In [4]:
import autograd.numpy as np
from autograd import grad

In [5]:
class ORNeuron():
    def __init__(self, n):
        self.n = n
        self.weights = np.random.rand(n)
        
    def getWeights(self):
        return self.weights    
    
    def present(self, inputs):
        z = np.sum(np.multiply(inputs, self.weights))
        return 1 - np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [6]:
class ANDNeuron():
    def __init__(self, n):
        self.weights = np.random.rand(n)
        
    def getWeights(self):
        return self.weights
        
    def present(self, inputs):
        i = 1.0 - inputs
        i[0] = 1.0
        
        z = np.sum(np.multiply(i, self.weights))
        return np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [7]:
class DNFNetwork():
    def __init__(self, n, c):
        self.n = n
        self.c = c
        
        d = []
        for i in range(0, c):
            d.append(ANDNeuron(n*2 + 1))
        
        self.conjunctions = np.array(d)
        
        if not c == 0:
            self.disjunction = ORNeuron(c + 1)
        else:
            self.disjunction = ORNeuron(n*2 + 1)
        
    def getDisjunction(self):
        return self.disjunction
    
    def getConjunctions(self):
        return self.conjunctions
    
    def fowardprop(self, inputs):
        actualIn = self.__convertInputs__(inputs)
        
        if not self.c == 0:
            dout = [1]
            for d in self.conjunctions:
                dout.append(d.present(actualIn))
            
            actualIn = dout
            
        return self.disjunction.present(np.array(actualIn))
    
    def __convertInputs__(self, inputs):
        actual = [1]
        
        for i in inputs:
            actual.append(i)
            actual.append(1-i)
        
        return np.array(actual)
    
    def __repr__(self):
        s = "Conjunctions -> "
        for d in self.conjunctions:
            s += (str(d) + ", ")
            
        s += ("\nDisjunction -> " + str(self.disjunction) + "\n")
            
        return s

In [8]:
def SSE(network, data, targets, p=False):
    predictions = predict(network, data)
    
#     if p:
#         print(predictions)
    
    return np.sum(np.power(np.subtract(targets, predictions), 2.0))
    
def predict(network, data):
    t = np.array([network.fowardprop(d) for d in data])
    return t
    

In [9]:
def trainDNFNetwork(data, targets, inputNodes, numC, it=10000, lr=0.1):
    network = DNFNetwork(inputNodes, numC)
    print(network)
    print("Initial Loss: ", SSE(network, data, targets, True))
    
    pterb = 0.0001
    
    for i in range(1, it):
        if i%1000 == 0:
            print("Iteration -> " + str(i) + " : " + str(SSE(network, data, targets)))
            
        for d in network.getConjunctions():
            g = computeNeuronGrad(network, d, pterb, data, targets)
            d.setGrad(g * lr)
            
        g = computeNeuronGrad(network, network.getDisjunction(), pterb, data, targets)
        
        network.getDisjunction().updateWeights(g * lr)
        for d in network.getConjunctions():
            d.applyGrad()
            
    print("Trained Loss: ", SSE(network, data, targets, True))
    return network

def computeNeuronGrad(network, neuron, pterb, data, targets):
    gradient = np.zeros(len(neuron.getWeights()))
    for k in range(0, len(neuron.getWeights())):
        g = np.zeros(len(neuron.getWeights()))
        g[k] = -pterb

        oldSSE = SSE(network, data, targets)
        neuron.updateWeights(g)
        newSSE = SSE(network, data, targets)
        neuron.updateWeights(-g)
                
        gradient[k] = (newSSE - oldSSE)/pterb
        
    return gradient

# NOT

In [10]:
dataNOT = np.array([[1.0], [0.0]])
targetsNOT = np.array([0.0, 1.0])
trainDNFNetwork(dataNOT, targetsNOT, 1, 0)

Conjunctions -> 
Disjunction -> [ 0.34971638  0.47671763]

Initial Loss:  0.468174751486
Iteration -> 1000 : 0.00255550496567
Iteration -> 2000 : 0.00126576353968
Iteration -> 3000 : 0.000840833798146
Iteration -> 4000 : 0.000629417685302
Iteration -> 5000 : 0.000502926583436
Iteration -> 6000 : 0.000418755712167
Iteration -> 7000 : 0.000358713130872
Iteration -> 8000 : 0.000313725938633
Iteration -> 9000 : 0.000278763070742
Trained Loss:  0.000250810257184


Conjunctions -> 
Disjunction -> [ 0.          4.14528646]

# AND
For this we dont neeed any disjunctions

In [11]:
dataAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsAND = np.array([0.0, 0.0, 0.0, 1.0])
trainDNFNetwork(dataAND, targetsAND, 2, 1)

Conjunctions -> [ 0.67524429  0.82908423  0.59884532  0.26151093], 
Disjunction -> [ 0.66211634]

Initial Loss:  0.733235577764
Iteration -> 1000 : 0.014361665406
Iteration -> 2000 : 0.00657302649531
Iteration -> 3000 : 0.00418802338239
Iteration -> 4000 : 0.00305123267219
Iteration -> 5000 : 0.00239081121983
Iteration -> 6000 : 0.00196090584645
Iteration -> 7000 : 0.00165950585428
Iteration -> 8000 : 0.00143685808519
Iteration -> 9000 : 0.00126586929347
Trained Loss:  0.00113055144934


Conjunctions -> [ 5.33176166  0.          5.33167858  0.        ], 
Disjunction -> [ 3.85498973]

# OR

In [12]:
dataOR = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsOR = np.array([0.0, 1.0, 1.0, 1.0])
trainDNFNetwork(dataOR, targetsOR, 2, 0)

Conjunctions -> 
Disjunction -> [ 0.13090894  0.86050171  0.57310611  0.96353061]

Initial Loss:  0.949547283017
Iteration -> 1000 : 0.00529712866834
Iteration -> 2000 : 0.00257887405663
Iteration -> 3000 : 0.00170304840819
Iteration -> 4000 : 0.00127101806606
Iteration -> 5000 : 0.00101373340023
Iteration -> 6000 : 0.000843033158886
Iteration -> 7000 : 0.000721514550327
Iteration -> 8000 : 0.000630603240704
Iteration -> 9000 : 0.000560031097652
Trained Loss:  0.000503660461708


Conjunctions -> 
Disjunction -> [ 4.14290141  0.          4.14348122  0.        ]

# NAND

In [13]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 1.0, 1.0, 0.0])
trainDNFNetwork(dataNAND, targetsNAND, 2, 2)

Conjunctions -> [ 0.64171082  0.43376487  0.40529173  0.9580646 ], [ 0.88923648  0.8740357   0.45839689  0.79283883], 
Disjunction -> [ 0.71163048  0.98976163]

Initial Loss:  0.733577449722
Iteration -> 1000 : 0.0203823208502
Iteration -> 2000 : 0.00869579609739
Iteration -> 3000 : 0.00543299376684
Iteration -> 4000 : 0.00392035480254
Iteration -> 5000 : 0.00305432719455
Iteration -> 6000 : 0.00249571800261
Iteration -> 7000 : 0.00210654414517
Iteration -> 8000 : 0.00182037602956
Iteration -> 9000 : 0.00160137372014
Trained Loss:  0.00142853554136


Conjunctions -> [ 0.          0.          0.          5.68376305], [ 0.         5.6763126  0.         0.       ], 
Disjunction -> [ 3.95341719  3.94885333]

# NOR

In [14]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 0.0, 0.0, 0.0])
trainDNFNetwork(dataNAND, targetsNAND, 2, 1)

Conjunctions -> [ 0.15205754  0.74725226  0.45301883  0.84495817], 
Disjunction -> [ 0.8828049]

Initial Loss:  0.815703445604
Iteration -> 1000 : 0.0141330108765
Iteration -> 2000 : 0.00652111132194
Iteration -> 3000 : 0.00416616708119
Iteration -> 4000 : 0.00303937087298
Iteration -> 5000 : 0.00238341471076
Iteration -> 6000 : 0.00195587160465
Iteration -> 7000 : 0.00165586659795
Iteration -> 8000 : 0.00143410893489
Iteration -> 9000 : 0.0012637217351
Trained Loss:  0.00112882896973


Conjunctions -> [ 0.          5.33265658  0.          5.33270657], 
Disjunction -> [ 3.85576644]

# XOR

In [15]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([0.0, 1.0, 1.0, 0.0])
trainDNFNetwork(dataNAND, targetsNAND, 2, 2)

Conjunctions -> [ 0.28515535  0.9984187   0.28713999  0.0147803 ], [ 0.0443784   0.1627957   0.85362651  0.60441   ], 
Disjunction -> [ 0.83370639  0.6395919 ]

Initial Loss:  1.11856733996
Iteration -> 1000 : 0.042381630163
Iteration -> 2000 : 0.0158265949478
Iteration -> 3000 : 0.00947380492484
Iteration -> 4000 : 0.00669615998895
Iteration -> 5000 : 0.00515347821174
Iteration -> 6000 : 0.00417683805077
Iteration -> 7000 : 0.00350502331592
Iteration -> 8000 : 0.00301556495362
Iteration -> 9000 : 0.00264361212448
Trained Loss:  0.00235168925604


Conjunctions -> [ 0.          5.66123891  5.66230458  0.        ], [ 5.63493607  0.          0.          5.63609754], 
Disjunction -> [ 3.84149539  3.8330935 ]

# IMPLIES

In [16]:
dataIMPLYS = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsIMPLYS = np.array([1.0, 1.0, 0.0, 1.0])
trainDNFNetwork(dataIMPLYS, targetsIMPLYS, 2, 2)

Conjunctions -> [ 0.92762178  0.74723084  0.82329901  0.52798605], [ 0.56419037  0.29670784  0.5215163   0.64670516], 
Disjunction -> [ 0.65578238  0.8353529 ]

Initial Loss:  0.801452834999
Iteration -> 1000 : 0.0252903217958
Iteration -> 2000 : 0.00865738647693
Iteration -> 3000 : 0.00529899847186
Iteration -> 4000 : 0.0038248584522
Iteration -> 5000 : 0.00298801749851
Iteration -> 6000 : 0.00244800178317
Iteration -> 7000 : 0.00207088042524
Iteration -> 8000 : 0.00179283729137
Iteration -> 9000 : 0.00157952640688
Trained Loss:  0.00141081120629


Conjunctions -> [ 0.          0.          5.66663643  0.        ], [ 0.          5.70299334  0.          0.        ], 
Disjunction -> [ 3.94208533  3.98211456]

# 3OR

In [17]:
data3OR = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targets3OR = np.array([0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
trainDNFNetwork(data3OR, targets3OR, 3, 3)

Conjunctions -> [ 0.07459963  0.68774625  0.766645    0.26554315  0.81294702  0.49648961], [ 0.99621017  0.52863637  0.89206277  0.43079252  0.84762626  0.60465774], [ 0.11000533  0.8178977   0.41860085  0.2825127   0.11027626  0.99678715], 
Disjunction -> [ 0.67104618  0.34862329  0.62978442]

Initial Loss:  1.20817645143
Iteration -> 1000 : 0.128256976716
Iteration -> 2000 : 0.0193558100619
Iteration -> 3000 : 0.00962738108176
Iteration -> 4000 : 0.0063543018228
Iteration -> 5000 : 0.0047180361903
Iteration -> 6000 : 0.00373980795576
Iteration -> 7000 : 0.00309092551934
Iteration -> 8000 : 0.00263064204556
Iteration -> 9000 : 0.0022930441829
Trained Loss:  0.00203429632008


Conjunctions -> [ 0.          0.          0.          0.          5.93385491  0.        ], [ 5.89643883  0.          0.          0.          0.          0.        ], [ 0.          0.25020642  5.80731052  0.          0.          0.53561631], 
Disjunction -> [ 4.42995525  3.94392556  3.88890387]

# 3AND

In [18]:
data3AND = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targets3AND = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
trainDNFNetwork(data3AND, targets3AND, 3, 1)

Conjunctions -> [ 0.59065129  0.4760426   0.1563      0.42635342  0.96851225  0.05086042], 
Disjunction -> [ 0.33788163]

Initial Loss:  1.21525613603
Iteration -> 1000 : 0.0213217368821
Iteration -> 2000 : 0.00956649622599
Iteration -> 3000 : 0.00603680103303
Iteration -> 4000 : 0.00437209880251
Iteration -> 5000 : 0.00341163046256
Iteration -> 6000 : 0.00278950530856
Iteration -> 7000 : 0.00235499954926
Iteration -> 8000 : 0.00203499937153
Iteration -> 9000 : 0.00178986073404
Trained Loss:  0.00159627111704


Conjunctions -> [ 5.29497867  0.          5.29488132  0.          5.29507229  0.        ], 
Disjunction -> [ 3.74791997]

# Complicated Formula

## (a AND b) OR (NOT c)

In [19]:
dataInteresting1 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting1 = np.array([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0])
trainDNFNetwork(dataInteresting1, targetsInteresting1, 3, 2)

Conjunctions -> [ 0.91990375  0.73032488  0.29522916  0.66688734  0.3335586   0.49088265], [ 0.36828615  0.50081645  0.45397834  0.9016962   0.43723996  0.63695044], 
Disjunction -> [ 0.32828048  0.95514621]

Initial Loss:  1.87638430608
Iteration -> 1000 : 0.0368890943724
Iteration -> 2000 : 0.0117828528827
Iteration -> 3000 : 0.00681469035443
Iteration -> 4000 : 0.00476576280874
Iteration -> 5000 : 0.00365295385594
Iteration -> 6000 : 0.0029568925369
Iteration -> 7000 : 0.00248742116719
Iteration -> 8000 : 0.00214959432065
Iteration -> 9000 : 0.00189396246443
Trained Loss:  0.00169352127657


Conjunctions -> [ 5.46040129  0.          5.45982407  0.          0.          0.        ], [ 0.          0.          0.          0.          0.          6.63753251], 
Disjunction -> [ 3.82081175  4.85255895]

## NOT ( a AND b AND c)

In [20]:
dataInteresting2 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting2 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0])
trainDNFNetwork(dataInteresting2, targetsInteresting2, 3, 1)

Conjunctions -> [ 0.82964757  0.44193089  0.89828838  0.8291308   0.75368753  0.66273073], 
Disjunction -> [ 0.23147714]

Initial Loss:  3.27104384604
Iteration -> 1000 : 0.623472325245
Iteration -> 2000 : 0.400745490465
Iteration -> 3000 : 0.358421934946
Iteration -> 4000 : 0.33491803788
Iteration -> 5000 : 0.318780281646
Iteration -> 6000 : 0.306567552403
Iteration -> 7000 : 0.296788317751
Iteration -> 8000 : 0.288662207207
Iteration -> 9000 : 0.281730267394
Trained Loss:  0.275700003588


Conjunctions -> [ 0.          1.15597069  0.          1.15597069  0.          1.15597069], 
Disjunction -> [ 16.53459183]

## a AND b AND c

In [21]:
dataInteresting3 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting3 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
trainDNFNetwork(dataInteresting3, targetsInteresting3, 3, 1)

Conjunctions -> [ 0.4752801   0.61059528  0.73604971  0.67945475  0.41662598  0.92047737], 
Disjunction -> [ 0.82660456]

Initial Loss:  2.25620610309
Iteration -> 1000 : 0.0219678394667
Iteration -> 2000 : 0.00970663636651
Iteration -> 3000 : 0.00609463608121
Iteration -> 4000 : 0.00440312454895
Iteration -> 5000 : 0.00343082681538
Iteration -> 6000 : 0.00280249699807
Iteration -> 7000 : 0.00236435042995
Iteration -> 8000 : 0.00204203862079
Iteration -> 9000 : 0.00179534386694
Trained Loss:  0.00160065835351


Conjunctions -> [ 5.29319961  0.          5.29329071  0.          5.29322458  0.        ], 
Disjunction -> [ 3.7465021]

# (a XOR b) AND c
CNF: ((NOT a) OR (NOT b)) AND (a OR b) AND c

In [22]:
dataInteresting4 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting4 = np.array([0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0])
trainDNFNetwork(dataInteresting4, targetsInteresting4, 3, 2, 10000, 0.5)

Conjunctions -> [ 0.29739807  0.00171116  0.80872767  0.98389067  0.70136888  0.63615846], [ 0.83480938  0.9396207   0.8704515   0.43406091  0.52096317  0.12775017], 
Disjunction -> [ 0.89989084  0.20539313]

Initial Loss:  2.23382902917
Iteration -> 1000 : 0.00704233471095
Iteration -> 2000 : 0.00324880568732
Iteration -> 3000 : 0.00208699831827
Iteration -> 4000 : 0.00152982710424
Iteration -> 5000 : 0.00120431817661
Iteration -> 6000 : 0.000991426354204
Iteration -> 7000 : 0.000841580403191
Iteration -> 8000 : 0.000730515340014
Iteration -> 9000 : 0.000644972894769
Trained Loss:  0.000577105155933


Conjunctions -> [ 6.69795023  0.          0.          6.69845327  6.35571326  0.        ], [ 0.          6.69694943  6.69741559  0.          6.35529502  0.        ], 
Disjunction -> [ 4.62845045  4.62810307]

# (a XOR b) OR c
DNF: (a AND (NOT b)) OR ((NOT a) AND b) OR c

In [23]:
dataInteresting5 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting5 = np.array([0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0])
trainDNFNetwork(dataInteresting5, targetsInteresting5, 3, 3)

Conjunctions -> [ 0.68860081  0.70957728  0.75320185  0.45308893  0.10868544  0.66358426], [ 0.75225144  0.45597351  0.38769003  0.05668319  0.6017873   0.385083  ], [ 0.48765209  0.62444084  0.74741854  0.02076235  0.95761706  0.89693144], 
Disjunction -> [ 0.74263656  0.8845297   0.03681781]

Initial Loss:  1.67382836333
Iteration -> 1000 : 0.360424732395
Iteration -> 2000 : 0.254228126441
Iteration -> 3000 : 0.213452674104
Iteration -> 4000 : 0.189924301583
Iteration -> 5000 : 0.173966574432
Iteration -> 6000 : 0.162150818196
Iteration -> 7000 : 0.152904749206
Iteration -> 8000 : 0.145389091784
Iteration -> 9000 : 0.139108313629
Trained Loss:  0.133746524113


Conjunctions -> [ 0.          6.85516584  6.85502313  0.          0.          0.        ], [ 2.11149332  0.          0.          2.11149266  2.3611198   0.        ], [ 0.3891798   0.72383336  0.75946156  0.00963957  1.01894747  0.83652127], 
Disjunction -> [  3.96095708  17.56061324   0.        ]

# (a XOR b) In 3Space
DNF: (a AND (NOT b)) OR ((NOT a) AND b)

In [24]:
dataInteresting4 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting4 = np.array([0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0])
trainDNFNetwork(dataInteresting4, targetsInteresting4, 3, 2)

Conjunctions -> [ 0.62165063  0.80569411  0.11778182  0.01865325  0.19892772  0.39363558], [ 0.72936251  0.15238085  0.44948014  0.7802324   0.68677952  0.33853165], 
Disjunction -> [ 0.77183791  0.61513605]

Initial Loss:  2.06746367866
Iteration -> 1000 : 0.0294632669614
Iteration -> 2000 : 0.0129589611416
Iteration -> 3000 : 0.00817835491121
Iteration -> 4000 : 0.00593767814757
Iteration -> 5000 : 0.00464568128227
Iteration -> 6000 : 0.00380796915521
Iteration -> 7000 : 0.00322199947623
Iteration -> 8000 : 0.00278973325792
Iteration -> 9000 : 0.00245804505223
Trained Loss:  0.00219569058586


Conjunctions -> [ 0.          6.12324249  6.12232565  0.          0.          0.        ], [ 6.12947793  0.          0.          6.1285871   0.          0.        ], 
Disjunction -> [ 4.22327547  4.22517956]

# (a OR b) in 3Space

In [25]:
dataOR3Space = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsOR3Space = np.array([0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
trainDNFNetwork(dataInteresting4, targetsInteresting4, 3, 1)

Conjunctions -> [ 0.37279682  0.76341437  0.99682985  0.23573099  0.32492959  0.83711015], 
Disjunction -> [ 0.18525496]

Initial Loss:  3.86934697577
Iteration -> 1000 : 1.3749287896
Iteration -> 2000 : 1.35121796238
Iteration -> 3000 : 1.34456459221
Iteration -> 4000 : 1.34147669192
Iteration -> 5000 : 1.33970364512
Iteration -> 6000 : 1.33855634656
Iteration -> 7000 : 1.33775462106
Iteration -> 8000 : 1.33716346009
Iteration -> 9000 : 1.3367099139
Trained Loss:  1.3363511605


Conjunctions -> [ 0.          7.89768712  7.89774781  0.          0.          0.        ], 
Disjunction -> [ 3.63998885]